# Iterative Template Generation

This notebook exemplifies one way in which a template mesh atlas can be generated from a collection of segmented binary images. Each binary image of a mouse femur is downsampled to reduce pixel density prior to applying marching cubes to generate a mesh from the binary image. One arbitrary mesh is selected as the template and then registered to and resampled from each original mesh to get a full set of meshes with correspondence points. The meshes are then groupwise registered via procrustes alignment and the mean mesh is taken as the new template. This process is repeated for a fixed number of iterations to get a template mesh atlas that represents the average case of all meshes.

This pipeline uses the [ITKShape](https://github.com/slicersalt/ITKShape) module for shape analysis.

In [ ]:
import sys
!{sys.executable} -m pip install numpy itk itk-shape itkwidgets

In [2]:
import os
import glob
import time

import itk
import numpy as np
from itkwidgets import view, checkerboard

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

from src.hasi.hasi import align
from src.hasi.hasi.pointsetentropyregistrar import PointSetEntropyRegistrar

### Read images

Input images represent the results of automatic binary segmentations of mouse femur data. Each image contains only the femur object and potentially represents a different region in space.

In [3]:
IMAGE_FOLDER = 'Input/femurs/'
DENSE_MESH_OUTPUT_FOLDER = 'Output/femurs/'
TEMPLATE_OUTPUT_FOLDER = 'Output/templates/'
MEAN_OUTPUT_FOLDER = 'Output/mean/'

for folder in [IMAGE_FOLDER, DENSE_MESH_OUTPUT_FOLDER, TEMPLATE_OUTPUT_FOLDER, MEAN_OUTPUT_FOLDER]:
    os.makedirs(folder, exist_ok=True)

In [4]:
# Get healthy femur segmentation binary images at 
# https://data.kitware.com/#collection/5dcc6691e3566bda4b802172/folder/5e0b8d6baf2e2eed35c326f7

input_paths = glob.glob(IMAGE_FOLDER + '*-R-*')
assert(len(input_paths) == 14)

print(input_paths)

['Input/femurs\\901-R-femur-label.nrrd', 'Input/femurs\\902-R-femur-label.nrrd', 'Input/femurs\\906-R-femur-label.nrrd', 'Input/femurs\\907-R-femur-label.nrrd', 'Input/femurs\\908-R-femur-label.nrrd', 'Input/femurs\\915-R-femur-label.nrrd', 'Input/femurs\\916-R-femur-label.nrrd', 'Input/femurs\\917-R-femur-label.nrrd', 'Input/femurs\\918-R-femur-label.nrrd', 'Input/femurs\\F9-3wk-01-R-femur-label.nrrd', 'Input/femurs\\F9-3wk-02-R-femur-label.nrrd', 'Input/femurs\\F9-3wk-03-R-femur-label.nrrd', 'Input/femurs\\F9-8wk-01-R-femur-label.nrrd', 'Input/femurs\\F9-8wk-02-R-femur-label.nrrd']


In [5]:
MESH_FILENAMES = [os.path.basename(file).replace('.nrrd','.obj') for file in input_paths]

In [6]:
images = list()
for path in input_paths:
    images.append(itk.imread(path, itk.UC))

### Paste images into same space

Here we standardize physical space across the femur images. This is primarily intended to assist in viewing convenience with `itkwidgets` which expects a standard viewing region, but could also be helpful to standardize output from subsequent image downsampling and mesh conversion operations.

In [7]:
images = align.paste_to_common_space(images)

In [8]:
view(images[1])

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

### Downsample images

The marching cubes algorithm returns a mesh with vertex density related to the pixel density of the original image. Here we downsample each image twice, once to get a "dense" image retaining most information density and a second time to get a "sparse" image more easily applied to get correspondence points.

Meshes generated later from the dense images have approximately 600,000 vertices each while meshes generated from the sparse images have approximately 4,000 vertices. We will use the dense meshes to sample feature information and iteratively refine a the atlas to generalize the shape population. We can select a single sparse mesh to act as the initial atlas or carry out iterative refinement on multiple sparse meshes and compare to determine which result "best" reflects the population.

In [9]:
SPARSE_DOWNSAMPLE_RATIO = 14
DENSE_DOWNSAMPLE_RATIO = 2

In [10]:
sparse_downsampled_images = align.downsample_images(images,SPARSE_DOWNSAMPLE_RATIO)
dense_downsampled_images = align.downsample_images(images,DENSE_DOWNSAMPLE_RATIO)

print(itk.size(dense_downsampled_images[0]))
print(itk.size(sparse_downsampled_images[0]))

itkSize3 ([639, 477, 519])
itkSize3 ([91, 68, 74])


In [11]:
view(dense_downsampled_images[0])

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageUC3; pr…

### Generate Meshes
The `itk.BinaryMask3DMeshSource` class makes use of the Marching Cubes algorithm to generate a mesh from a given object. Each binary image here uses the value '1' to indicate the femur is present at a pixel and '0' to indicate the femur is not present. Marching Cubes rapidly fills the femur space and generates surfaces at pixel region boundaries.

Note that it may be useful to visually examine intermediate results. In the case where a mesh is not well aligned with others it is useful to correct the transformation in an external mesh editor.

In [12]:
# Here each pixel interior to the femur has value "1" and exterior has value "0".
# This may change for a different segmentation image.
FEMUR_OBJECT_PIXEL_VALUE = 1

In [13]:
dense_meshes = align.binary_image_list_to_meshes(dense_downsampled_images,
                                           object_pixel_value=FEMUR_OBJECT_PIXEL_VALUE)

# Expect ~200K vertices
print('Average dense mesh points: ' +
      str(sum([mesh.GetNumberOfPoints() for mesh in dense_meshes]) / len(dense_meshes)))

Average dense mesh points: 215237.7857142857


In [14]:
sparse_meshes = align.binary_image_list_to_meshes(sparse_downsampled_images,
                                            object_pixel_value=FEMUR_OBJECT_PIXEL_VALUE)

# Expect ~200K vertices
print('Average sparse mesh points: ' +
      str(sum([mesh.GetNumberOfPoints() for mesh in sparse_meshes]) / len(sparse_meshes)))

Average sparse mesh points: 4371.5


In [15]:
# Write out each mesh to disk
for i in range(0,len(dense_meshes)):
    itk.meshwrite(dense_meshes[i], f'{DENSE_MESH_OUTPUT_FOLDER}{MESH_FILENAMES[i]}')

for i in range(0,len(sparse_meshes)):
    itk.meshwrite(sparse_meshes[i], f'{TEMPLATE_OUTPUT_FOLDER}{MESH_FILENAMES[i]}')

In [16]:
# visualize with itkwidgets
view(geometries=[mesh for mesh in sparse_meshes])

Template itk::ImageToPointSetFilter<itk::Image<signedshort,2>,itk::PointSet<signedshort,2>>
 already defined as <class 'itk.itkImageToPointSetFilterPython.itkImageToPointSetFilterISS2PSSS2'>
 is redefined as {cl}
Template itk::ImageToPointSetFilter<itk::Image<signedshort,3>,itk::PointSet<signedshort,3>>
 already defined as <class 'itk.itkImageToPointSetFilterPython.itkImageToPointSetFilterISS3PSSS3'>
 is redefined as {cl}
Template itk::ImageToPointSetFilter<itk::Image<signedshort,4>,itk::PointSet<signedshort,4>>
 already defined as <class 'itk.itkImageToPointSetFilterPython.itkImageToPointSetFilterISS4PSSS4'>
 is redefined as {cl}
Template itk::ImageToPointSetFilter<itk::Image<unsignedchar,2>,itk::PointSet<unsignedchar,2>>
 already defined as <class 'itk.itkImageToPointSetFilterPython.itkImageToPointSetFilterIUC2PSUC2'>
 is redefined as {cl}
Template itk::ImageToPointSetFilter<itk::Image<unsignedchar,3>,itk::PointSet<unsignedchar,3>>
 already defined as <class 'itk.itkImageToPointSetFi

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'numberOfComponents': 3, 'd…

### Register Sample for Correspondence
Our goal is to find correspondence points between the template and each sample mesh. In order to get correspondence, two steps are employed:
- First, a copy of the template mesh is registered to the target sample;
- Second, each mesh point is repositioned at its nearest neighbor on the target sample.

The result of this process is a full collection of meshes having the same number of points and correspondence between each point such that it represents the same approximate feature on each femur. The cells below show an example of the template being registered to a single sample mesh. This process is repeated for every sample in the full pipeline.

In [17]:
registered_template = align.register_template_to_sample(sparse_meshes[1],
                                                  dense_meshes[2],
                                                  learning_rate=1.0,
                                                  minimum_convergence_value=-0.1,
                                                  max_iterations=500,
                                                  verbose=True)

Iteration: 0 Metric: 0.0
Iteration: 0 Metric: 0.0
Iteration: 0 Metric: 0.0
Iteration: 0 Metric: 0.0
Iteration: 0 Metric: 0.0
Iteration: 0 Metric: 0.7779061854942507
Iteration: 1 Metric: 0.20232000292614374
Iteration: 2 Metric: 0.13674457929331374
Iteration: 3 Metric: 0.12514542842076454
Iteration: 4 Metric: 0.122550291731051
Iteration: 5 Metric: 0.12061420415595286
Iteration: 6 Metric: 0.11841144247855505
Iteration: 7 Metric: 0.11589176266252948
Iteration: 8 Metric: 0.11325031330727194
Iteration: 9 Metric: 0.11063534863433273
Iteration: 10 Metric: 0.10807184441782834
Iteration: 11 Metric: 0.10561519395469256
Iteration: 12 Metric: 0.10323751692873938
Iteration: 13 Metric: 0.10090638903838738
Iteration: 14 Metric: 0.09868776219709642
Iteration: 15 Metric: 0.09658259228964905
Iteration: 16 Metric: 0.09458964512549776
Iteration: 17 Metric: 0.09270177175503178
Iteration: 18 Metric: 0.09092719465045711
Iteration: 19 Metric: 0.08921088449330158
Iteration: 20 Metric: 0.08751331057883362
Iterat

Iteration: 199 Metric: 0.050829382864870494
Iteration: 200 Metric: 0.05082170914386962
Iteration: 201 Metric: 0.05081423866472783
Iteration: 202 Metric: 0.05080568023614978
Iteration: 203 Metric: 0.05079902262172653
Iteration: 204 Metric: 0.05079059523282873
Iteration: 205 Metric: 0.05078300527810398
Iteration: 206 Metric: 0.05077665585338589
Iteration: 207 Metric: 0.05076873757269327
Iteration: 208 Metric: 0.05076176368430392
Iteration: 209 Metric: 0.05076191191460448
Iteration: 210 Metric: 0.050759255781437584
Iteration: 211 Metric: 0.05075297240683363
Iteration: 212 Metric: 0.05074280619842757
Iteration: 213 Metric: 0.0507353605396458
Iteration: 214 Metric: 0.0507293774424416
Iteration: 215 Metric: 0.05072597680923839
Iteration: 216 Metric: 0.050725369103616885
Iteration: 217 Metric: 0.0507235730125808
Iteration: 218 Metric: 0.05071929349275945
Iteration: 219 Metric: 0.05071276157849241
Iteration: 220 Metric: 0.050704833611047065
Iteration: 221 Metric: 0.05070007184266171
Iteration:

Iteration: 390 Metric: 0.05018866379176612
Iteration: 391 Metric: 0.0501866580990529
Iteration: 392 Metric: 0.05018544461556436
Iteration: 393 Metric: 0.050183944161473114
Iteration: 394 Metric: 0.05018252535440755
Iteration: 395 Metric: 0.050181952039632355
Iteration: 396 Metric: 0.05018052191351881
Iteration: 397 Metric: 0.05017906303736656
Iteration: 398 Metric: 0.050178842342465696
Iteration: 399 Metric: 0.05017926878989274
Iteration: 400 Metric: 0.0501761496165269
Iteration: 401 Metric: 0.05017516704134721
Iteration: 402 Metric: 0.05017371936068065
Iteration: 403 Metric: 0.05017193665910303
Iteration: 404 Metric: 0.050170784743674424
Iteration: 405 Metric: 0.050170514887143504
Iteration: 406 Metric: 0.050171765973031336
Iteration: 407 Metric: 0.050170869118313775
Iteration: 408 Metric: 0.05016937934734629
Iteration: 409 Metric: 0.0501673844188967
Iteration: 410 Metric: 0.050166100196711874
Iteration: 411 Metric: 0.05016514666894262
Iteration: 412 Metric: 0.05016612976804814
Iterat

In [18]:
view(geometries=[registered_template, dense_meshes[2]])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'numberOfComponents': 3, 'd…

### Resample Template for Correspondence

With the template mesh registered to the sample we can find correspondence points on the surface of the sample bone by getting the nearest sample point to every point on the template mesh surface.

In [19]:
sample_correspondence = align.resample_template_from_target(registered_template, dense_meshes[2])
view(geometries=[sample_correspondence,dense_meshes[2]])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'numberOfComponents': 3, 'd…

### Define Procrustes Alignment Parameters
Once template meshes have been aligned to represent each input mesh with correspondence we can run Procrustes alignment and get out a mean mesh as the new template. Here we demonstrate aligning two sample meshes as an example, but the full population is used in the atlas generation pipeline.

In [20]:
second_registered_template = align.register_template_to_sample(sparse_meshes[1],
                                                               dense_meshes[3],
                                                               max_iterations=100)
second_sample_correspondence = align.resample_template_from_target(second_registered_template, dense_meshes[3])

mean_correspondence = align.get_mean_correspondence_mesh([sample_correspondence,second_sample_correspondence],
                                                        convergence_threshold=1.0,
                                                        verbose=False)
view(geometries=[mean_correspondence])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'numberOfComponents': 3, 'd…

### Compute Hausdorff Distance

We can calculate the Hausdorff distance from the current to previous mesh atlas at each iterative refinement to quantify the amount of change between iterations. In this case the meshes are in correspondence so we get the largest Euclidean distance between any pair of correspondence points.

In [21]:
dist = align.get_pairwise_hausdorff_distance(mean_correspondence,sparse_meshes[1])
print(f'Hausdorff distance between initial and updated mesh: {dist}')

view(geometries=[sparse_meshes[1],mean_correspondence])

Hausdorff distance between initial and updated mesh: 2.018958098608886


Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'numberOfComponents': 3, 'd…

### Run Iterative Refinement

Here we run the previously described registration, deformation, and alignment steps on every sample mesh, iterating the template as we go.

In [22]:
# Fix the number of iterative atlas refinements.
# For the mouse femur data set the atlas tends to converge
# after 3-5 iterations.
NUM_ITERATIONS = 1

# Select index of atlas template to refine.
# Initial template should be as close to a "mean" shape
# as possible, with no distinct surface outliers.
TEMPLATE_IDX = 1

# Prepare directory to write out atlas iterations.
# ex. 'Output/mean/0/901-L-femur-label.obj'
for i in range(0,NUM_ITERATIONS):
    os.makedirs(MEAN_OUTPUT_FOLDER + str(i) + '\\', exist_ok=True)

In [23]:
template_mesh = sparse_meshes[TEMPLATE_IDX]
for iteration in range(0, NUM_ITERATIONS):
    starttime = time.time()
    updated_template = align.refine_template_from_population(template_mesh=template_mesh,
                                                             target_meshes=dense_meshes,
                                                             registration_iterations=200)
    endtime = time.time()
    print(f'Time elapsed for iteration {iteration}: {endtime - starttime}')
    
    output_path = f'{MEAN_OUTPUT_FOLDER}/{iteration}/{MESH_FILENAMES[TEMPLATE_IDX]}'
    print(f'Writing to {output_path}')
    itk.meshwrite(updated_template, output_path)
    
    template_mesh = updated_template

Time elapsed for iteration 0: 164.92673254013062
Writing to Output/mean//0/902-R-femur-label.obj


The final template may be further examined via point distance metrics to determine fitness as an atlas. Here we will simply view the result.

In [24]:
# Visualize 
view(geometries=[template_mesh])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'numberOfComponents': 3, 'd…